In [1]:
import sys
import os

# Get the project root absolute path
project_root = os.path.abspath("..")  # Move up one level to project root

# Add project root to sys.path
if project_root not in sys.path:
    sys.path.append(project_root)

print("Project root added to sys.path:", project_root)

Project root added to sys.path: /Users/matth/OneDrive/Documents/DukeMIDS/DataPlus/Basketball/DL_homography/homography_deep_learning_model


In [2]:
import numpy as np
from model.reprojection_loss import ReprojectionLoss
from dataset.point_selection import sample_evenly_spaced_points
import torch

In [3]:
gparent_directory = os.path.abspath("../..")
homography_directory = os.path.join(gparent_directory, "DL_homography_matrices", "OFFENSE-1")
mask_directory = os.path.join(gparent_directory, "DL_masks", "OFFENSE-1")
print(homography_directory)

/Users/matth/OneDrive/Documents/DukeMIDS/DataPlus/Basketball/DL_homography/DL_homography_matrices/OFFENSE-1


In [4]:
# One test homography matrix 
files = sorted(os.listdir(homography_directory))
file = "Frame_0.npy"

# Load H_gt [1,3,3] and image mask [224,224]
mask = np.load(os.path.join(mask_directory, file))
H_gt = np.array([np.load(os.path.join(homography_directory, file))])
H_gt = torch.tensor(H_gt, dtype=torch.float32)

# Generate valid points for the mask
points = np.array([sample_evenly_spaced_points(mask=mask, num_points=10, seed = 42)])
points = torch.tensor(points, dtype=torch.float32) 
print(points)
print(points.shape)


tensor([[[180., 194.,   1.],
         [ 87., 163.,   1.],
         [201.,  80.,   1.],
         [104.,  23.,   1.],
         [195.,  81.,   1.],
         [170.,  71.,   1.],
         [173., 160.,   1.],
         [177., 126.,   1.],
         [102., 165.,   1.],
         [148.,  98.,   1.]]])
torch.Size([1, 10, 3])


In [5]:
reprojection_loss = ReprojectionLoss()

In [6]:
print(H_gt.shape)
H_gt.transpose(1, 2).shape

torch.Size([1, 3, 3])


torch.Size([1, 3, 3])

In [7]:
error = reprojection_loss.generate_reprojection(H_pred=H_gt, H_gt=H_gt, points=points)
print(error)

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [23]:
def generate_perturbed_matrix(H, total_mse):
    errors = torch.randn(9, dtype=H.dtype, device=H.device)  # Random errors for 9 elements
    errors *= torch.sqrt(torch.tensor(total_mse) / torch.sum(errors**2))  # Normalize to total MSE
    
    perturbed_H = H.clone()
    perturbed_H += errors.reshape(3, 3)  # Apply errors to the whole matrix
    
    # Optionally re-normalize h33 to 1
    # perturbed_H /= perturbed_H[2, 2]
    return perturbed_H


In [16]:
H_gt[:2, :]

tensor([[[ 8.5514e-05, -1.7402e-05,  9.9974e-01],
         [ 7.0484e-06,  2.0510e-04, -2.2602e-02],
         [-5.0311e-08,  2.4724e-09,  4.3624e-04]]])

In [21]:
errors = np.random.randn(9)  # Random errors for 8 elements
errors *= np.sqrt(0.05 / np.sum(errors**2))
errors.shape

(9,)

In [20]:
errors.reshape(2, 4)[:2, :3]

array([[ 0.08040371,  0.14968848, -0.01912936],
       [-0.08489319,  0.07019564,  0.07660192]])

In [78]:
# Iterate through different MSE values
mse_values = np.linspace(0.0001, 10, 1)
reprojection_errors = []

for mse in mse_values:
    H_perturbed = generate_perturbed_matrix(H_gt, mse)
    error = reprojection_loss.generate_reprojection(H_pred=H_perturbed, H_gt=H_gt, points=points)
    reprojection_errors.append(torch.mean(error))
    print("MSE: ", torch.nn.functional.mse_loss(H_perturbed, H_gt))

print(reprojection_errors)

MSE:  tensor(1.1111e-05)
[tensor(40928400.)]


In [31]:
np.linspace(0.0000001, 0.1, 10)

array([1.00000e-07, 1.11112e-02, 2.22223e-02, 3.33334e-02, 4.44445e-02,
       5.55556e-02, 6.66667e-02, 7.77778e-02, 8.88889e-02, 1.00000e-01])

array([1.e-05])